In [7]:
%%capture
# may take a while to build font cache
%matplotlib inline

In [8]:
import radd
from radd import build, vis

# Dependent Process (SS Race) Model

In [54]:
# Initial state of Stop process (red) depends on current strength of 
# Go activation (green) assumes Stop signal efficacy at later SSDs diminishes 
# as the state of the Go process approaches the execution threshold (upper bound). 
# (pink lines denote t=SSD, blue is trial deadline)
radd.load_dpm_animation()

In [53]:
# read data into pandas DataFrame (http://pandas.pydata.org/)
# example_data contains data from 15 subjects in the 
# Reactive Stop-Signal task discussed in Dunovan et al., (2015)
data = radd.load_example_data()
data.head()

,idx,Cond,ttype,choice,response,acc,rt,ssd
0,28,bsl,go,go,1,1,0.59853,1000
1,28,bsl,go,go,1,1,0.52017,1000
2,28,bsl,go,go,1,1,0.54513,1000
3,28,bsl,go,go,1,1,0.57159,1000
4,28,bsl,go,go,1,1,0.50522,1000


# Formatting data for radd
## Required columns
* **idx**: Subject ID number
* **ttype**: Trial-Type ('go' if no stop-signal, 'stop' if stop-signal trial)
* **response**: Trial Response (1 if response recorded, 0 if no response)
* **acc**: Choice Accuracy (1 if correct, 0 if error)
* **rt**: Choice Response-Time (in seconds, can be any value no-response trials)
* **ssd**: Stop-Signal Delay (in milliseconds, 1000 on go trials)

## Optional columns
* input dataframe can contain columns for experimental conditions of interest (choose any name)
* in the dataframe above, the **Cond** column contains **'bsl'** and **'pnl'**  
    * in the **'bsl'** or **"Baseline"** condition, errors on **go** and **stop** trials are equally penalized 
    * in the  **'pnl'** or **"Caution"** condition, penalties are doubled for **stop** trial errors  (e.g., response=1)
* See below for fitting models with conditional parameter dependencies 
    * e.g., drift-rate depends on levels of 'Cond'

# Building a model

In [12]:
model = build.Model(data=data, kind='xdpm', depends_on={'v':'Cond'})
model.observedDF.head()

,idx,Cond,acc,200,250,300,350,400,c10,c20,...,c90,e10,e20,e30,e40,e50,e60,e70,e80,e90
0,28,bsl,0.99174,1.0,1.0,0.95,0.60,0.00,0.50509,0.52523,...,0.59818,0.49608,0.51848,0.53166,0.53177,0.53187,0.54489,0.54585,0.55842,0.56744
1,28,pnl,0.97521,1.0,1.0,0.95,0.80,0.10,0.51767,0.53183,...,0.61186,0.51983,0.53243,0.54525,0.55420,0.55858,0.56381,0.57181,0.58508,0.60212
2,29,bsl,0.99174,1.0,1.0,1.00,0.90,0.00,0.52504,0.53773,...,0.59844,0.52676,0.54505,0.54510,0.54888,0.55845,0.55854,0.57092,0.58484,0.59019
3,29,pnl,0.96694,1.0,1.0,1.00,0.75,0.35,0.53145,0.54520,...,0.62498,0.53144,0.53222,0.54477,0.54504,0.55171,0.56291,0.57507,0.58511,0.59804
4,30,bsl,0.94215,1.0,1.0,1.00,0.80,0.25,0.52978,0.55850,...,0.63843,0.53607,0.54522,0.56057,0.58415,0.58535,0.59848,0.60980,0.61176,0.62076


## Header of observed dataframe (model.observedDF)
* **idx**: subject ID
* **Cond**: Baseline(bsl)/Caution(pnl) (could be any experimental condition of interest) 
* **Acc**: Accuracy on "go" trials
* **sacc**: Mean accuracy on "stop" trials (mean condition SSD used during simulations)
* **c10 - c90**: 10th - 90th RT quantiles for correct responses
* **e10 - e90**: 10th - 90th RT quantiles for error responses

# Bounded Global & Local Optimization

## Global Optimization (Basinhopping w/ bounds)
tams()** method gives control over low-level parameters used for global opt
* **xtol = ftol = tol**: error tolerance of global optimization (default=1e-20)

* **stepsize** (default=.05): set basinhopping initial step-size
    * see HopStep class in radd.fit for more details
    * see get_stepsize_scalars() in radd.theta for parameter-specific step-sizes

* **nsamples** (default=3000): number of parameter subsets to sample
    * number of individual parameter subsets $\theta_i \in \Theta$ to sample and evaluate before initializing global opt
    
    $$\Theta = \{\theta_1, \theta_2 \dots \theta_{nsamples}\}$$
    
    * each sampled parameter subset $\theta_i = \{a, v, tr, \dots\, xb\}$ (see table below for description of parameters) is used to simulate a theoretical data vector $\hat{Y}_{\theta_i} = f_{sim}(\theta_i)$ which is then compared to the empirical data vector $Y$ via the weighted cost function:
    
    * The parameter set (or *sets* - see **ninits** below) that yield the lowest cost function error ($\varepsilon_{\theta_i}$) are then used to initialize the model for global optimization
   
   
$$\varepsilon_{\theta_i} = \sum [\omega * (\hat{Y}_{\theta_i} - Y)]^2$$
   
$$P_{inits} = \operatorname*{argmin}_{\varepsilon}(\varepsilon_{\theta_i}\: \vert \: \theta_i\,\in \Theta)$$ 


| $$\theta$$ | Description     | str id | Go/Stop |
|:----------:|:----------------|:------:|:-------:|
| $$a_{G}$$  | Threshold       |  'a'   | Go      |
| $$tr_{G}$$ | Onset-Delay     |  'tr'  | Go      |
| $$v_{G}$$  | Drift-Rate      |  'v'   | Go      |  
| $$xb_{G}$$ | Dynamic Gain    |  'xb'  | Go      |
| $$v_{S}$$  | SS Drift-Rate   |  'ssv' | Stop    |
| $$so_{S}$$ | SS Onset-Delay  |  'sso' | Stop    |


* **ninits** (default=3): number of initial parameter sets to perform global optimization on

    * if ninits is 1 global optimization is performed once, using sampled parameter set with the lowest cost error (as described above in **nsamples**)

    * if ninits is greater than 1 then global optimization is performed $n$ separate times, one for each each $p_{i} \in P_{inits}$ where $P_{inits}$ is the rank ordered set of parameters subsets corresponding to $n-{th}$ lowest cost error
    
    * The optimized parameters corresponding to the lowest global minimum across all iterations of basinhopping are then selected and passed to the next stage in the fitting routine (local gradient-based optimization)

* **nsuccess** (default=60): criterion number of successful steps without finding new global minimum to exit basinhopping

* **interval** (default=10): number of steps before adaptively updating the stepsize 

* **T** (default=1.0): set the basinhopping "temperature"
    * higher T will result in accepted steps with larger changes in function value (larger changes in model error)
    
### Using set_basinparams() to control global opt. params
```python
model.set_basinparams(tol=1e-15, nsamples=4000, ninits=6, nsuccess=70)
```

# Sampling Distributions for Init Parameters

In [16]:
# sample model.basinparams['nsamples'] from each dist (default=3000)
# and initialize model with best model.basinparams['ninits'] (default=3)
vis.plot_param_distributions(p=model.inits)

## Local Optimization (Nelder-Mead Simplex w/ bounds)
* **set_fitparams()** method gives control over low-level parameters used for local opt. Local optimization polishes parameter estimates passed from global optimization step.

 * **method** (default='nelder'): optimization algorithm 
     * (see [here](https://lmfit.github.io/lmfit-py/fitting.html#choosing-different-fitting-methods) for list of available methods)

 * **xtol = ftol = tol** (default=1e-30): error tolerance of optimization

 * **maxfev** (default=2000): max number of func evaluations to perform

 * **ntrials** (default=20000): num. simulated trials per condition

### Using set_fitparams() to control local optimization parameters

```python
model.set_fitparams(method='tnc', tol=1e-35, ntrials=30000, maxfev=2500)
```

### Using set_fitparams() to set/access low-level model attributes

* **set_fitparams()** also allows you to control low-level attributes of the model, including...

 * **quantiles** (default=np.array([.10, .20, ... .90]): quantiles of RT distribution

 * **kind** (default='dpm'): model kind (currently only irace and dpm) 

 * **depends_on** (dict): {parameter_id : condition_name}

 * **tb** (float): trial duration (timewindow for response) in seconds

 * **nlevels** (int): number of levels in depends_on data[condition_name]  

 * **fit_on (default='average')**: by default, models are fit to the 'average' 
    data (across subjects). If **fit_on='subjects'**, a model is fit to each individual 
    subject's data.
    
```python
q = np.arange(.1, 1.,.05)
model.set_fitparams(kind='irace', depends_on={'a': 'Cond'}, quantiles=q)
```

# Steps in Fitting Routine

### Step 1a: 
- Global optimzation on flat data (average values collapsing across experimental conditions)

### Step 1b:
- Local optimzation using parameters passed from global optimizer as starting values 

### Step 2a
- Global optimzation of conditional parameters 

### Step 2b:
- Local optimzation of conditional parameters passed from global optimizer


## Step 1. Flat Fits
- All models are initially fit by optimizing the full set of parameters to the "flattened" data (flat meaning the average data collapsing across all conditions of interest). For instance, at this stage fitting the dependent process model involves finding the parameter values for each included parameter that minimizes the cost-function cost function: 

    $$Cost = \sum [\omega * (\hat{Y} - Y)]^2$$
    
  
- *$Y$* is an array of observed data (e.g., accuracy, RT quantiles, etc.) 
- *$\hat{Y}$* is an equal length array of corresponding model-predicted values given parameters $\theta$
- The error between the predicted and the observed data (**$\hat{Y} - Y$**) is weighted by an equal length array of scalars **$\omega$** proportional to the inverse of the variance in each value of **$Y$**. 
- The array of weighted differences is summed, yielding a single cost value equal to the summed squared-error (**$SSE$**). 

### Optimizing flat models
* Because we didn't provide any conditional dependencies to the model above optimizing 
    the model will only go through Step 1 of the fitting routine (performing global optimization
    on all the model parameters). 
   
* This can take some time (~10-20m depending processing power, settings in fitparams, basinparams etc.)

* Step 2 is faster but owes it's speed to the heavy lifting done at this stage


### How to access...

|model information | method used to calculate | how to access|
|:--|:--|:--|
| flat data ($Y$) | **model**.observedDF.mean() | **model**.observed_flat |
| flat weights ($\omega$) | **model**.wtsDF.mean() | **model**.flat_wts |

In [17]:
# build a model with no conditional dependencies
model = build.Model(data=data, kind='xdpm')
# set progress to True to track global and local error basins during fit
model.optimize(progress=True, plotfits=True)

## Step 2. Conditional Fits
- Conditional models can be fit in which all parameters from **Step 1** are held constant except for one or more designated **conditional** parameters which is free to vary across levels of an experimental condition of interest. Global and local optimization are performed by minimizing the cost-function: 

$$Cost = \sum_{i=0}^{N_c} [\omega_i * (\hat{Y_i} - Y_i)]^2$$
    
- where $\sum[\omega_i*(\hat{Y_i} - Y_i)]^2$ gives the **$Cost$** for level **$i$** of condition **$C$** 
- the total **$Cost$** is equal to the **$SSE$** across all **$N_c$** levels of that condition
- Specifying parameter dependencies is done using **depends_on** --> **{parameter_id : condition_name}**.
- For instance, in Dunovan et al., (2015) subjects performed two versions of a stop-signal task 
     * **Baseline ("bsl")** condition: errors on **go** and **stop** trials are equally penalized 
     * **Caution ("pnl")** condition: penalties 2x higher for **stop** trial errors (e.g., response=1)
- To test the hypothesis that observed behavioral differences between penalty conditions was a result of a change Go drift-rate...

``` py
# define the model allowing Go drift-rate to vary across 'Cond'
model = build.Model(kind='xdpm', depends_on={'v': 'Cond'})
# run optimize to fit the full model (steps 1 & 2)
model.optimize(progress=True)
```

## How to access...
|model information | method used to calculate | how to access|
|:--|:--|:--|
|conditional data ($Y_i$) | **model**.observedDF.groupby(**cond**).mean()| **model**.observed |
|conditional weights ($\omega_i$) | **model**.wtsDF.groupby(**cond**).mean() |  **model**.cond_wts |

# Typically...

* ...you'll have multiple competing hypotheses about which parameters will depend on various task conditions

* For instance, one Hypothesis might be that assymetric stop/go penalties alter behavior by changing the height of the decision threshold **(a)**....

``` py
# define the model allowing threshold to vary across 'Cond'
model_a = build.Model(kind='xdpm', depends_on={'a': 'Cond'})
# run optimize to fit the full model (steps 1 & 2)
model_a.optimize(progress=True)
```

- Alternatively, behavioral effects of penalty might be driven by a change in drift-rate **(v)**, rather than the height of the decision threshold.

```py
# define alt model allowing drift-rate to vary across 'Cond'
model_v = build.Model(data, depends_on={'v':'Cond'})
model_v.optimize(progress=True)
```

## compare model fits
```python
# Is threshold model better? 
model_a.finfo['AIC'] < model_v.finfo['AIC']
```

## How to access...

|model information | method used to calculate | how to access|
|:--|:--|:--|
|conditional data ($Y_i$) | **model**.observedDF.groupby(**cond**).mean()| **model**.observed |
|conditional weights ($\omega_i$) | **model**.wtsDF.groupby(**cond**).mean() |  **model**.cond_wts |

In [47]:
# build conditional model and optimize with drift-rate free across levels of Cond
model = build.Model(data=data, kind='xdpm', depends_on={'v':'Cond'})
model.optimize(progress=True)

# Nested optimization of alternative models

* Typically you'll have multiple competing hypotheses about which parameters are influenced by various task conditions

* Nested optimization allows alternative models to be optimized using a single initial parameter set.

```python 
model = build.Model(kind='xdpm', data=data)
free_params = ['v', 'a', 'tr']
model.nested_optimize(free=free_params, cond='Cond')
```

* After fitting the model with **depends_on={'v': 'Cond'}**, **'v'** is replaced by the next parameter in the **nested_models** list **('a' or boundary height in this case)** and the model is optimized with this dependency using the same init params as the original model

* As a result, model selection is less likely to be biased by the initial parameter set

* Also, because Step 1 takes significantly longer than Step 2, nested optimization of multiple models is significantly faster than optimizing each model individually through boths steps

* Run the cell below and go take a leisurely coffee break

In [48]:
model = build.Model(kind='xdpm', data=data)
nested_models = ['v', 'a', 'tr']
model.nested_optimize(free=nested_models, cond='Cond', progress=True)

# Examine Nested Model Fits

## Parameter estimates and GOF stats

In [49]:
# get rid of any extra rows in the fitDF 
model.fitDF

,idx,a,ssv,tr,v,xb,bsl,pnl,nfev,nvary,df,chi,rchi,logp,AIC,BIC,cnvrg
0,v,0.59775,-1.1952,0.11453,"[0.934995901191,...",1.9731,0.935,0.90144,1545,2,46,0.0041934,9.116e-05,-446.54,-442.54,-441.97,False
1,a,"[0.585008262263,...",-1.1952,0.11453,0.92204,1.9731,0.58501,0.61601,213,2,46,0.0055081,0.00011974,-433.45,-429.45,-428.88,True
2,tr,0.59775,-1.1952,"[0.11000000514, ...",0.92204,1.9731,0.11,0.12002,1476,2,46,0.0049296,0.00010716,-438.78,-434.78,-434.21,False


In [50]:
# Evaluate all nested fits and plot fit stats: AIC & BIC (Lower is better)
gof = vis.compare_nested_models(model.fitDF, verbose=True, model_ids=nested_models)

AIC likes v model
BIC likes v model


## Model Predictions (same header as model.observedDF)

In [51]:
# model predictions
# to save as csv file: model_v.yhatDF.to_csv("save_path", index=False)
# to extract values as numpy ndarray: model_v.yhatDF.loc[:, 'acc':].values
model.yhatDF

,idx,Cond,acc,200,250,300,350,400,c10,c20,...,c90,e10,e20,e30,e40,e50,e60,e70,e80,e90
0,v,bsl,0.98900,1.0,0.9995,0.9505,0.5100,0.0885,0.52453,0.52953,...,0.60453,0.50953,0.52453,0.52953,0.53953,0.54453,0.55453,0.55953,0.57453,0.58953
1,v,pnl,0.97465,1.0,1.0000,0.9735,0.6220,0.1420,0.52953,0.53953,...,0.61453,0.52453,0.52953,0.53953,0.54453,0.55453,0.55953,0.57453,0.57953,0.59453
2,a,bsl,0.99065,1.0,1.0000,0.9550,0.5280,0.1015,0.51953,0.53453,...,0.60453,0.51453,0.51953,0.53453,0.53953,0.54953,0.55453,0.56453,0.56953,0.58453
3,a,pnl,0.98325,1.0,1.0000,0.9735,0.6185,0.1300,0.52953,0.54453,...,0.61453,0.52453,0.52953,0.54453,0.55453,0.55953,0.56453,0.57453,0.57953,0.59453
4,tr,bsl,0.99020,1.0,1.0000,0.9540,0.5205,0.0930,0.52000,0.53500,...,0.60500,0.51500,0.52000,0.53500,0.54000,0.55000,0.55500,0.56500,0.57000,0.58500
5,tr,pnl,0.98570,1.0,1.0000,0.9790,0.6225,0.1375,0.53002,0.54502,...,0.61502,0.52502,0.53002,0.53502,0.55002,0.55002,0.56502,0.57502,0.58002,0.59502


#  Testing model identity (parameter recovery)
* **recover_model()** samples a large number of parameter sets and chooses one that yields a synthetic data vector close to the actual observed data vectors in **model.observed_flat** & **model.observed**
* the model then goes through the fitting routine, optimizing an independent set of parameters to the synthetic data
* when the model is finished optimizing, the recovered parameters are compared and plotted against the generative set of parameters. 
* if the values are close, this is evidence that the model is identifiable 

In [ ]:
# build a model and sample parameters
model = build.Model(data=data, kind='xdpm', depends_on={'v':'Cond'})
# fit synthetic data and compare fitted vs. init params
model.recover_model(progress=True, plotparams=True)

# Troubleshooting Ugly Fits

## Fit to individual subjects
```python
model = build.Model(data=data, fit_on='subjects')
```
## Other "kinds" of models...


* Currently only Dependent Process Model **(kind='dpm')** and Independent Race Model **(kind='irace')**


* Tell model to include a Dynamic Bias Signal **('xb')** by adding an **'x'** to the front of model **kind**


* To implement the **Dependent Process Model**...

```python
#... with dynamic bias: 
model = build.Model(data=data, kind='xdpm')
#...and without: 
model = build.Model(data=data, kind='dpm')
```


* To implement the **Independent Race Model**... 


```python
#... with dynamic bias:
model = build.Model(data=data, kind='xirace')
#... and without:
model = build.Model(data=data, kind='irace')
```



## Optimization parameters and cost weights...

* set more conservative fit criteria

```python
model.set_basinparams(nsuccess=100, tol=1e-30, ninits=10, nsamples=10000) 
model.set_fitparams(maxfev=5000, tol=1e-35)
```

* Inspect cost function weights for extreme vals

```python
print(model.cond_wts)
print(model.flat_wts)
```
* If the wts look suspect try re-running the fits with an unweighted model (all wts = 1)     


```python
 model = build.Model(data=data, weighted=False)
```

* Keep in mind that error RTs can be particularly troublesome, sometimes un-shootably so...

In [19]:
import radd
radd.style_notebook()

Notebook Theme: Grade3
(https://www.github.com/dunovank/jupyter-themes)
